In [5]:
!pip install chromadb google-generativeai pillow numpy pandas sentence-transformers python-dotenv

## Setup and Imports

In [8]:
import os
import base64
import io
import json
from typing import List, Dict, Any, Optional
import numpy as np
from PIL import Image
import google.generativeai as genai
import chromadb
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer
import pandas as pd


## Entire RAG configuration


In [9]:
from dotenv import load_dotenv
import os

load_dotenv('/content/drive/MyDrive/api_keys.env')

hf_token = os.getenv("HF_TOKEN")
gemini_api = os.getenv("GEMINI_API_KEY")

class Config:
    """Configuration class for the RAG system"""

    def __init__(self):
        # API Configuration
        self.gemini_api_key = gemini_api

        # ChromaDB Configuration
        self.chroma_db_path = "./chroma_db"
        self.collection_name = "cat_health_multimodal"

        self.text_model = "gemini-1.5-pro"
        self.vision_model = "gemini-1.5-pro"
        self.embedding_model = "all-MiniLM-L6-v2"

        self.max_retrieved_docs = 5
        self.similarity_threshold = 0.3

config = Config()

#Gemini Setup
genai.configure(api_key=config.gemini_api_key)

# Initialize models
text_model = genai.GenerativeModel(config.text_model)
vision_model = genai.GenerativeModel(config.vision_model)

# Initialize embedding model for text
embedding_model = SentenceTransformer(config.embedding_model)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


APIs and models configured successfully!


## ChromaDB Setup


In [10]:
class MultimodalVectorStore:
    """Handles vector storage and retrieval for multimodal data"""

    def __init__(self, db_path: str, collection_name: str):
        self.client = chromadb.PersistentClient(path=db_path)
        self.embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(
            model_name=config.embedding_model
        )

        # Create or get collection
        self.collection = self.client.get_or_create_collection(
            name=collection_name,
            embedding_function=self.embedding_function,
            metadata={"description": "Multimodal cat health knowledge base"}
        )

    def add_text_document(self, text: str, metadata: Dict[str, Any], doc_id: str):
        """Add text document to the vector store"""
        self.collection.add(
            documents=[text],
            metadatas=[metadata],
            ids=[doc_id]
        )

    def add_image_document(self, image_description: str, image_analysis: str,metadata: Dict[str, Any], doc_id: str):
        """Add image document (as text description) to the vector store"""
        combined_text = f"Image Description: {image_description}\nAnalysis: {image_analysis}"
        metadata.update({"type": "image_analysis"})

        self.collection.add(
            documents=[combined_text],
            metadatas=[metadata],
            ids=[doc_id]
        )

    def search(self, query: str, n_results: int = 5) -> Dict[str, Any]:
        """Search for similar documents"""
        results = self.collection.query(
            query_texts=[query],
            n_results=n_results
        )
        return results

    def get_collection_stats(self):
        """Get statistics about the collection"""
        return {
            "total_documents": self.collection.count(),
            "collection_name": self.collection.name
        }

# Initialize vector store
vector_store = MultimodalVectorStore(config.chroma_db_path, config.collection_name)
print(f"Vector store initialized: {vector_store.get_collection_stats()}")


Vector store initialized: {'total_documents': 0, 'collection_name': 'cat_health_multimodal'}


## Image Processing Pipeline

In [11]:
def encode_image_to_base64(image_path: str) -> str:
    """Convert image file to base64 string"""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def pil_image_to_base64(image: Image.Image) -> str:
    """Convert PIL Image to base64 string"""
    buffered = io.BytesIO()
    image.save(buffered, format="JPEG")
    return base64.b64encode(buffered.getvalue()).decode('utf-8')

def analyze_cat_image(image_path: str) -> Dict[str, str]:
    """Analyze cat image using Gemini Vision API"""

    # Load and prepare image
    image = Image.open(image_path)

    # Prepare prompt for cat health analysis
    prompt = """
    You are a veterinary expert analyzing this cat image. Please provide:

    1. Physical Description: Describe the cat's appearance, breed characteristics, and any visible features
    2. Health Assessment: Note any visible signs that might indicate health issues including:
       - Eyes (clarity, discharge, color)
       - Nose (discharge, color)
       - Coat condition (shine, thickness, patches)
       - Body posture and positioning
       - Visible wounds, swelling, or abnormalities
       - Behavioral indicators visible in the image
    3. Potential Concerns: List any potential health concerns based on visual cues
    4. Recommendations: Suggest when veterinary consultation might be needed

    Be thorough but note that this is not a substitute for professional veterinary examination.
    """

    try:
        response = vision_model.generate_content([prompt, image])
        return {
            "analysis": response.text,
            "status": "success"
        }
    except Exception as e:
        return {
            "analysis": f"Error analyzing image: {str(e)}",
            "status": "error"
        }

## RAG System Setup

In [15]:
class CatHealthRAG:
    """Main RAG system for cat health analysis"""

    def __init__(self, vector_store: MultimodalVectorStore):
        self.vector_store = vector_store

    def add_knowledge_base_entry(self, content: str, source_type: str,
                                category: str, doc_id: str):
        """Add entry to the knowledge base"""
        metadata = {
            "source_type": source_type,
            "category": category,
            "timestamp": pd.Timestamp.now().isoformat()
        }

        self.vector_store.add_text_document(content, metadata, doc_id)

    def add_cat_image_analysis(self, image_path: str, doc_id: str,
                              additional_info: str = ""):
        """Add cat image analysis to knowledge base"""

        # Analyze image
        analysis_result = analyze_cat_image(image_path)

        if analysis_result["status"] == "success":
            # Create description for embedding
            image_desc = f"Cat image analysis with additional context: {additional_info}"

            metadata = {
                "source_type": "image_analysis",
                "category": "visual_assessment",
                "image_path": image_path,
                "timestamp": pd.Timestamp.now().isoformat()
            }

            self.vector_store.add_image_document(
                image_desc,
                analysis_result["analysis"],
                metadata,
                doc_id
            )

            return analysis_result["analysis"]
        else:
            print(f"Failed to analyze image: {analysis_result['analysis']}")
            return None

    def query_multimodal(self, query_text: str, image_path: Optional[str] = None,
                        max_results: int = 5) -> str:
        """Query the RAG system with text and optionally an image"""

        # Prepare context from query
        context_parts = [f"User Query: {query_text}"]

        # If image provided, analyze it
        if image_path:
            image_analysis = analyze_cat_image(image_path)
            if image_analysis["status"] == "success":
                context_parts.append(f"Current Image Analysis: {image_analysis['analysis']}")

                # Use image analysis to enhance search query
                enhanced_query = f"{query_text} {image_analysis['analysis'][:200]}"
            else:
                enhanced_query = query_text
        else:
            enhanced_query = query_text

        # Search knowledge base
        search_results = self.vector_store.search(enhanced_query, max_results)

        # Prepare retrieved context
        retrieved_docs = []
        if search_results['documents'] and search_results['documents'][0]:
            for i, doc in enumerate(search_results['documents'][0]):
                metadata = search_results['metadatas'][0][i]
                retrieved_docs.append(f"Source {i+1} ({metadata.get('category', 'unknown')}): {doc}")

        system_prompt = """
        You are an expert veterinary assistant helping with cat health analysis.
        Use the provided context from the knowledge base and any image analysis to give
        comprehensive, helpful advice about potential cat health issues.

        IMPORTANT: Always recommend consulting with a veterinarian for proper diagnosis
        and treatment. Your role is to provide initial insights and guidance.
        """

        user_prompt = f"""
        {system_prompt}

        Context from Knowledge Base:
        {chr(10).join(retrieved_docs) if retrieved_docs else "No relevant documents found."}

        Current Query and Analysis:
        {chr(10).join(context_parts)}

        Please provide a comprehensive response addressing:
        1. Analysis of the situation based on available information
        2. Possible health concerns or issues
        3. Immediate care recommendations if applicable
        4. When to seek veterinary care
        5. Preventive measures or monitoring suggestions

        Response:
        """

        try:
            response = text_model.generate_content(user_prompt)
            return response.text
        except Exception as e:
            return f"Error generating response: {str(e)}"

# Initialize RAG system
rag_system = CatHealthRAG(vector_store)


##Populating the Knowledge Base

In [17]:
def initialize_knowledge_base():
    """Initialize knowledge base with sample cat health information"""

    sample_knowledge = [
        {
            "content": "Common eye issues in cats include conjunctivitis (pink eye), which causes redness, discharge, and squinting. Clear discharge usually indicates allergies or irritation, while thick yellow or green discharge suggests bacterial infection requiring antibiotic treatment.",
            "category": "eye_health",
            "doc_id": "eye_health_001"
        },
        {
            "content": "Upper respiratory infections in cats often present with nasal discharge, sneezing, and congestion. Clear nasal discharge may indicate viral infection, while thick colored discharge suggests secondary bacterial infection. Cats may also show reduced appetite and lethargy.",
            "category": "respiratory_health",
            "doc_id": "respiratory_001"
        },
        {
            "content": "Healthy cat coat should be shiny, smooth, and free of bald patches. Dull, brittle, or patchy fur can indicate nutritional deficiencies, stress, parasites, or underlying health issues. Excessive shedding or matting may also signal problems.",
            "category": "coat_health",
            "doc_id": "coat_health_001"
        },
        {
            "content": "Signs of pain or discomfort in cats include changes in posture (hunched back, reluctance to move), hiding behavior, changes in eating habits, vocalization changes, and altered grooming patterns. Cats often hide pain well, so subtle changes are important.",
            "category": "pain_indicators",
            "doc_id": "pain_001"
        },
        {
            "content": "Dental issues in cats can cause bad breath, drooling, difficulty eating, and pawing at the mouth. Yellow or brown tartar buildup on teeth and red, swollen gums indicate dental disease requiring professional cleaning and treatment.",
            "category": "dental_health",
            "doc_id": "dental_001"
        },
        {
            "content": "Skin conditions in cats may present as red, inflamed areas, excessive scratching, hair loss, or unusual lumps. Common causes include allergies, parasites (fleas, mites), bacterial or fungal infections, and autoimmune conditions.",
            "category": "skin_health",
            "doc_id": "skin_001"
        }
    ]

    for item in sample_knowledge:
        rag_system.add_knowledge_base_entry(
            content=item["content"],
            source_type="reference_material",
            category=item["category"],
            doc_id=item["doc_id"]
        )

    print(f"Knowledge base initialized with {len(sample_knowledge)} entries")

# Initialize the knowledge base
initialize_knowledge_base()

Knowledge base initialized with 6 entries


## Some Examples and Usage

In [18]:
def example_usage():
    """Demonstrate how to use the RAG system"""

    print("=" * 60)
    print("CAT HEALTH RAG SYSTEM - USAGE EXAMPLES")
    print("=" * 60)

    # Example 1: Text-only query
    print("\n1. TEXT-ONLY QUERY EXAMPLE")
    print("-" * 30)

    text_query = "My cat has been sneezing and has watery eyes. What could be wrong?"
    response = rag_system.query_multimodal(text_query)
    print(f"Query: {text_query}")
    print(f"Response: {response[:500]}...")

    # Example 2: Adding image analysis (you'll need to provide image path)
    print("\n2. IMAGE ANALYSIS EXAMPLE")
    print("-" * 30)
    print("To analyze an image, use:")
    print("image_path = '/path/to/your/cat/image.jpg'")
    print("analysis = rag_system.add_cat_image_analysis(image_path, 'my_cat_001', 'Cat seems lethargic')")
    print("response = rag_system.query_multimodal('What health issues do you see?', image_path)")

    # Example 3: Adding custom knowledge
    print("\n3. ADDING CUSTOM KNOWLEDGE")
    print("-" * 30)
    custom_knowledge = "My cat Whiskers tends to hide under the bed when feeling unwell. This is her specific stress response pattern."
    rag_system.add_knowledge_base_entry(
        content=custom_knowledge,
        source_type="personal_observation",
        category="behavioral_patterns",
        doc_id="whiskers_behavior_001"
    )
    print("Added custom knowledge about Whiskers' behavior patterns")

    # Show collection stats
    print(f"\n4. KNOWLEDGE BASE STATISTICS")
    print("-" * 30)
    stats = vector_store.get_collection_stats()
    print(f"Total documents in knowledge base: {stats['total_documents']}")


## Some Interactive and Intuitive functions

In [19]:
def analyze_single_image(image_path: str, additional_context: str = ""):
    """Analyze a single cat image"""

    print(f"Analyzing image: {image_path}")
    analysis = rag_system.add_cat_image_analysis(
        image_path,
        f"analysis_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}",
        additional_context
    )

    if analysis:
        print("Analysis completed:")
        print(analysis)
    else:
        print("Failed to analyze image")

def ask_health_question(question: str, image_path: str = None):
    """Ask a health question with optional image"""

    print(f"Question: {question}")
    if image_path:
        print(f"With image: {image_path}")

    response = rag_system.query_multimodal(question, image_path)
    print(f"Response:\n{response}")

    return response

def add_personal_knowledge(content: str, category: str):
    """Add personal knowledge about your cat"""

    doc_id = f"personal_{category}_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}"
    rag_system.add_knowledge_base_entry(
        content=content,
        source_type="personal_knowledge",
        category=category,
        doc_id=doc_id
    )
    print(f"Added personal knowledge to category: {category}")

In [21]:
if __name__ == "__main__":
    example_usage()

    print("\n" + "=" * 60)
    print("READY TO USE!")
    print("=" * 60)
    print("\nThe cat health RAG system is now functioning.")
    print("\nQuick start guide:")
    print("1. Replace 'YOUR_GEMINI_API_KEY_HERE' with your actual Gemini API key")
    print("2. Use ask_health_question() for text queries")
    print("3. Use analyze_single_image() to analyze cat photos")
    print("4. Use add_personal_knowledge() to add information about your specific cat")

    print("\nExample commands:")
    print("ask_health_question('My cat is not eating well')")
    print("analyze_single_image('/path/to/cat/photo.jpg')")
    print("add_personal_knowledge('My cat loves hiding in boxes when sick', 'behavior')")


CAT HEALTH RAG SYSTEM - USAGE EXAMPLES

1. TEXT-ONLY QUERY EXAMPLE
------------------------------


Query: My cat has been sneezing and has watery eyes. What could be wrong?
Response: Error generating response: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits....

2. IMAGE ANALYSIS EXAMPLE
------------------------------
To analyze an image, use:
image_path = '/path/to/your/cat/image.jpg'
analysis = rag_system.add_cat_image_analysis(image_path, 'my_cat_001', 'Cat seems lethargic')
response = rag_system.query_multimodal('What health issues do you see?', image_path)

3. ADDING CUSTOM KNOWLEDGE
------------------------------
Added custom knowledge about Whiskers' behavior patterns

4. KNOWLEDGE BASE STATISTICS
------------------------------
Total documents in knowledge base: 7

READY TO USE!

The cat heatlth RAG system is now functioning.

## Some additional helper functions

In [22]:
def batch_analyze_images(image_folder: str):
    """Analyze multiple images in a folder"""
    import glob

    image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.gif', '*.bmp']
    image_files = []

    for ext in image_extensions:
        image_files.extend(glob.glob(os.path.join(image_folder, ext)))
        image_files.extend(glob.glob(os.path.join(image_folder, ext.upper())))

    results = {}
    for i, image_path in enumerate(image_files):
        print(f"Processing image {i+1}/{len(image_files)}: {os.path.basename(image_path)}")
        doc_id = f"batch_analysis_{i+1}_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}"
        analysis = rag_system.add_cat_image_analysis(image_path, doc_id)
        results[image_path] = analysis

    return results

def export_knowledge_base():
    """Export current knowledge base to JSON"""

    # Get all documents from ChromaDB
    all_docs = vector_store.collection.get()

    export_data = {
        "collection_name": vector_store.collection.name,
        "export_timestamp": pd.Timestamp.now().isoformat(),
        "documents": []
    }

    for i in range(len(all_docs['ids'])):
        doc_data = {
            "id": all_docs['ids'][i],
            "document": all_docs['documents'][i],
            "metadata": all_docs['metadatas'][i] if all_docs['metadatas'] else {}
        }
        export_data["documents"].append(doc_data)

    # Save to file
    export_filename = f"cat_health_kb_export_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.json"
    with open(export_filename, 'w') as f:
        json.dump(export_data, f, indent=2)

    print(f"Knowledge base exported to: {export_filename}")
    return export_filename

print("Your multimodal cat health RAG system is ready to use!")

Your multimodal cat health RAG system is ready to use!
